## Projeto 2: Construindo um dashboard
Vimos que é possível construir gráficos dinâmicos em aula.

Agora é a oportunidade de juntar o que aprendemos de pandas, seaborn, e plotly. Integrando num dashboard que será hospedado no streamlit.

### Objetivo:
- Construir um dashboard dinâmico com dados de interesse
    - Ações
    - Casos de COVID
    - Viagens da Uber
    - Qualquer tipo de dados!
- Aplicar boas práticas de desenvolvimento com pandas e gráficos
- Escolher a palheta de cor adequada
- Escolher gráficos adequados
- Criar uma estória para o usuário com os dados  

#### Objetivo extra
- Publicar o dash
Como publicar o dash?  
https://docs.streamlit.io/en/stable/deploy_streamlit_app.html



Não precisamos criar 10 gráficos distintos, ou um com muitas informações. As vezes um ou dois gráficos já é o suficiente para passar a mensagem de forma objetiva!

link do streamlit  
https://streamlit.io/  
A documentação é bem detalhada! Olhe com atenção e veja as possibilidades de botões, filtros e gráficos.  
https://docs.streamlit.io/en/stable/



### Dados:
`yfinance` modulo para extração de valores de ações.

https://data.world/datasets/uber

https://ourworldindata.org/


Trabalho: https://data.world/data-society/discover-the-menu/workspace/intro

In [74]:
import pandas as pd
# chamando os aqrquivos
dish = './Dish.csv'
menuitem = './MenuItem.csv'
menu = './Menu.csv'
menu_page = './MenuPage.csv'

dish_raw = pd.read_csv(dish)
menuitem_raw = pd.read_csv(menuitem)
menu_raw = pd.read_csv(menu)
menu_page_raw = pd.read_csv(menu_page)

# manipulação dos dados
menuitem_raw = menuitem_raw.drop(columns=['created_at','updated_at','xpos','ypos','high_price'])
menuitem_raw = menuitem_raw.dropna(subset=['price'])

dish_raw = dish_raw.drop(columns=['description','first_appeared','last_appeared'])
dish_raw = dish_raw.dropna(subset=['lowest_price','highest_price'])

menu_page_raw = menu_page_raw.drop(columns=['page_number','image_id','full_height','full_width','uuid'])

menu_raw = menu_raw.drop(columns=['status','notes','occasion','venue','event','keywords','language','location_type','name','page_count','call_number','physical_description','place'])
menu_raw = menu_raw.dropna(subset=['currency','sponsor','date'])

# aplicando os joins
df = menuitem_raw.set_index('dish_id').join(dish_raw.set_index('id')).reset_index(drop=True)
df_ = df.set_index('menu_page_id').join(menu_page_raw.set_index('id')).reset_index(drop=True)
df_final = df_.set_index('menu_id').join(menu_raw.set_index('id'),how='inner').reset_index(drop=True)

# fazendo ultimas manipulações
df_final = df_final[~(df_final.date == '2928-03-26')]
df_final = df_final[~(df_final['date'] <= '1091-01-27')]
df_final['date'] = pd.to_datetime(df_final['date'])
df_final['year'] = df_final.date.dt.year
df_final['price_var'] = df_final.apply(lambda x: x['highest_price'] - x['lowest_price'] if x['lowest_price'] > 0 else 0, axis=1)

# salvando no pdf
df_final.to_csv('df_final.csv',index=False)

In [76]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783610 entries, 0 to 784134
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   id               783610 non-null  int64         
 1   price            783610 non-null  float64       
 2   name             783450 non-null  object        
 3   menus_appeared   783450 non-null  float64       
 4   times_appeared   783450 non-null  float64       
 5   lowest_price     783450 non-null  float64       
 6   highest_price    783450 non-null  float64       
 7   sponsor          783610 non-null  object        
 8   date             783610 non-null  datetime64[ns]
 9   location         783610 non-null  object        
 10  currency         783610 non-null  object        
 11  currency_symbol  783610 non-null  object        
 12  dish_count       783610 non-null  int64         
 13  year             783610 non-null  int64         
 14  price_var        783